## Python Module and Packages Lecture and Lab

This interpreter essentionally reproduces lis.py from Peter Norvig. Go read his posts.

### Environment Interface

In [1]:
import abc
class Environment(abc.ABC):
    """
    This is the interface for an Environment. The client for 
    this interface is a language intepreter. 
    """
    @classmethod
    @abc.abstractmethod
    def empty(cls):
        return cls()
    
    @abc.abstractmethod
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
    
    @abc.abstractmethod
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        
    @abc.abstractmethod
    def lookup(variable):
        """
        return the unique binding of the variable and the environment it was bound
        in as a tuple. If it is not found raise a NameError as below
        """
        raise NameError("{} not found in Environment".format(vaiable))

### Environment Implementation

In [2]:
class Env:
    """
    Absfun: the dicionary {k1:v1, k2:v2,...} represents the
    environment binding k1 to v1 and k2 to v2. There are no duplicates.
    
    The keys k must be strings, and the values v must be legitimate values
    in our environment.
    
    The empty dictionary represents the empty environment.
    
    Repinv: Newer bindings replace older bindings in the dictionary. 
    This is guaranteed by using python dictionaries.
    """
    
    def __init__(self, outerenv=None):
        self.env = dict()
        self.outerenv = outerenv
    
    @classmethod
    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        self.env[variable] = value
        
    def extend_many(self, envdict):
        self.env.update(envdict)
        
    def lookup(self, key):
        try:
            found = self.env[key]
            env = self
        except KeyError:
            if self.outerenv is not None:
                found, env =self.outerenv.lookup(key)
            else:
                raise NameError("{} <<>> not found in Environment".format(key))
        return found, env
            
Environment.register(Env)

__main__.Env

### Parser

In [3]:
Symbol = str

def typer(token):
    if token == 'true':
        return True
    elif token == 'false':
        return False
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return Symbol(token)
        
def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token
    
def parse(loc):
    return syn(lex(loc))

### Evaluator

In [4]:
def global_env(envclass):
    "An environment with some Scheme standard procedures."
    import math, operator as op
    env = envclass.empty()
    env.extend_many(vars(math))
    env.extend_many({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '==':op.eq,
        'not':     op.not_
    })
    return env

In [5]:
class Function():
    def __init__(self, params, parsedbody, env):
        self.params = params
        self.code = parsedbody
        self.env = env
        self.envclass = env.__class__
        
    def __call__(self, *args):
        funcenv = self.envclass(outerenv = self.env)
        funcenv.extend_many(zip(self.params, args))
        return eval_ptree(self.code, funcenv)

In [6]:
def eval_ptree(x, env):
    fmap={'#t':True, '#f':False, 'nil':None}
    if x in ('#t', '#f', 'nil'):
        return fmap[x]        
    elif isinstance(x, Symbol):
        # variable lookup
        return env.lookup(x)[0]
    elif not isinstance(x, list):  # constant
        return x
    elif len(x)==0: #noop
        return None
    elif x[0]=='if':
        (_, predicate, truexpr, falseexpr) = x
        if eval_ptree(predicate, env):
            expression = truexpr
        else:
            expression = falseexpr
        return eval_ptree(expression, env)
    elif x[0] == 'def':         # variable definition
        (_, var, expression) = x
        #postorder traversal by nested eval is needed below
        # your code here
        env.extend(var, eval_ptree(expression, env))
    elif x[0] == 'store':           # (set! var exp)
        (_, var, exp) = x
        env.lookup(var)[1].extend(var, eval_ptree(exp, env))
    elif x[0] == 'func':
        (_, parameters, parsedbody) = x
        return Function(parameters, parsedbody, env)
    else:                          # operator
        op = eval_ptree(x[0], env)
        #postorder traversal to get subexpressione before running the op
        args = [eval_ptree(arg, env) for arg in x[1:]]
        return op(*args)

In [7]:
class Program():
    
    def __init__(self, program, env):
        self.program = [e.strip() for e in program.split('\n')]
        self.env = env
        
    def __iter__(self):
        for line in self.program:
            yield line
    
    def parse(self):
        for l in iter(self):
            yield parse(l)
            
    def run(self):
        for l in iter(self):
            yield eval_ptree(parse(l), self.env)

### Driver to run the code

In [8]:
def backtolang(exp):
    boolmap={True:'#t', False:'#f'}
    if  isinstance(exp, list):
        return '(' + ' '.join(map(backtolang, exp)) + ')' 
    elif isinstance(exp, bool):
        return boolmap[exp]
    elif exp is None:
        return 'nil'
    else:
        return str(exp)
    
def repl(env, prompt='calc> '):
    while True:
        try:
            val = eval_ptree(parse(input(prompt)), env)
        except (KeyboardInterrupt, EOFError):
            break
        if val is not None: 
            print(backtolang(val))
            


In [9]:
def run_program_asif_repl(program, env):
    prog=Program(prpgram, globenv)
    for result in p1c.run():
        print(backtolang(result))

In [10]:
def run_program(program, env):
    prog=Program(prpgram, globenv)
    endit = None
    for result in p1c.run():
        endit = result
    return endit

In [11]:
globenv = global_env(Env)

In [12]:
repl(globenv)# to get out of the repl in the notebook just cause an exception like below

calc> a


NameError: a <<>> not found in Environment

### Tests

In [13]:
p1 = """
(def ra 5)
ra
(if (== (> 2 3) #t) #f ra)
"""

In [14]:
program = """
(def rad 5)
rad
(def radiusfunc (func (radius) (* pi (* radius radius))))
(radiusfunc rad)
(def myvar 0)
(if (== myvar 1) (store rad 6) (store rad 7))
(radiusfunc rad)
(== 1 1)
"""

In [15]:
p=Program(program, globenv)
list(iter(p))

['',
 '(def rad 5)',
 'rad',
 '(def radiusfunc (func (radius) (* pi (* radius radius))))',
 '(radiusfunc rad)',
 '(def myvar 0)',
 '(if (== myvar 1) (store rad 6) (store rad 7))',
 '(radiusfunc rad)',
 '(== 1 1)',
 '']

In [16]:
for s in p.parse():
    print(s)

[]
['def', 'rad', 5]
rad
['def', 'radiusfunc', ['func', ['radius'], ['*', 'pi', ['*', 'radius', 'radius']]]]
['radiusfunc', 'rad']
['def', 'myvar', 0]
['if', ['==', 'myvar', 1], ['store', 'rad', 6], ['store', 'rad', 7]]
['radiusfunc', 'rad']
['==', 1, 1]
[]


In [17]:
for result in p.run():
    print(backtolang(result))

nil
nil
5
nil
78.53981633974483
nil
nil
153.93804002589985
#t
nil


### pyscaffold

Pyscaffold is a project which creates a scaffolding for us for a python project. We'll use it to understand the structure of a python project and the modules that go therein.

Get into your `py35` virtual environment.
We'll see:

- the notion of a package module (also see https://docs.python.org/3/tutorial/modules.html)
- generating documentation
- doing tests on the fly
- creating command line executables.
- installing in development mode (`pip install -e .`)
- uninstalling(`pip uninstall packagename`)

Create a "skeleton project" witht he incantation (mit license for example)

`putup -l mit --with-travis projectname`

In [100]:
!putup -h

usage: putup [-h] [-p NAME] [-d TEXT] [-u URL] [-l LICENSE] [-f] [-U]
             [--with-namespace NS1[.NS2]]
             [--with-cookiecutter TEMPLATE | --with-django] [--with-travis]
             [--with-pre-commit] [--with-tox] [-v]
             PROJECT

PyScaffold is a tool for easily putting up the scaffold of a Python project.

positional arguments:
  PROJECT               project name

optional arguments:
  -h, --help            show this help message and exit
  -p NAME, --package NAME
                        package name (default: project name)
  -d TEXT, --description TEXT
                        package description (default: '')
  -u URL, --url URL     package url (default: '')
  -l LICENSE, --license LICENSE
                        package license from dict_keys(['none', 'lgpl2',
                        'affero', 'gpl3', 'cc0', 'eclipse', 'public-domain',
                        'new-bsd', 'proprietary', 'artistic', 'apache',
                        'simple-bsd', 'isc', '

We'll now go look inside.

`python setup.py` incantations:


- python setup.py --help-commands
- python setup.py install
- pip install -e .
- python setup.py sdist
- python setup.py tests
- python setup.py doctest
- python setup.py docs
- python setup.py --help
- python setup.py build
- python setup.py clean
- python setup.py develop

### The stupidlang package

I used the template here to create a package for our language. Go get it here: https://github.com/rahuldave/stupidlang . As a lab, we'll play a game of documenting it, testing it, etc.

Also see http://python-packaging-user-guide.readthedocs.org/en/latest/distributing/#requirements-for-packaging-and-distributing.

What we'll learn?

- how to do command lines
- how relative imports work
- installing into virtual environments (`conda create py35-test`). Also see http://conda.pydata.org/docs/using/envs.html and http://docs.python-guide.org/en/latest/dev/virtualenvs/.
- capturing environments(`pip freeze > requirements.txt`, `pip install -r requirements.txt` ) and (`conda env export > environment.yml`, `conda env create -f environment.yml`)
- getting stuff working on travis and with coveralls.